In [1]:
# Assignment 0

In [26]:
import pandas as pd
import numpy as np
from numpy import linalg as LNG 

In [20]:
# Training inputs (image as vector)
X_train = pd.read_csv("data/train_in.csv").to_numpy()

# Training outputs (number that each vector represents)
Y_train = pd.read_csv("data/train_out.csv").to_numpy()

X_test = pd.read_csv("data/test_in.csv").to_numpy()
Y_test = pd.read_csv("data/test_out.csv").to_numpy()

In [44]:
ones = []

for x,y in zip(X_train, Y_train):
    if y == 1:
        ones.append(x)

ones

[array([-1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.51 ,
        -0.213, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.292,
         0.792, -0.987, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.56 ,
         0.975, -0.873, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.745,
         0.999, -0.748, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.596,
         1.   , -0.601, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.714,
         1.   , -0.585, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -0.998,  0.714,
         1.   , -0.454, -1.   , -1.   , -1.   , -1.   , -1.   , 

In [75]:
def get_centroid(X_train):
    sum = np.zeros(X_train.shape[1])
    for x in X_train:
        sum = sum + x
    return sum / X_train.shape[0]


In [74]:
get_centroid(np.array(ones))

(252, 256)


array([-1.        , -1.        , -1.        , -1.        , -1.        ,
       -0.99697222, -0.66161111,  0.51029365,  0.03004365, -0.90127381,
       -0.99891667, -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -0.9820754 , -0.48426587,  0.82638492,  0.71113492,
       -0.60656349, -0.99217857, -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -0.97902778, -0.45460317,  0.86704762,
        0.85275   , -0.48153175, -0.9740873 , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -0.9809246 , -0.44045635,
        0.86792857,  0.88074206, -0.38481746, -0.97134524, -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -0.97

In [79]:
inputs = np.array([
    [5,3],
    [1,6],
    [3,1],
    [2,0],
    [8,5]
])

inputs

array([[5, 3],
       [1, 6],
       [3, 1],
       [2, 0],
       [8, 5]])

In [80]:
get_centroid(inputs)

array([3.8, 3. ])

In [ ]:
label_1_sum = np.zeros((256))
len_1 = 0

for x,y in zip(X_train, Y_train):
    if y == 1:
        label_1_sum = label_1_sum + x
        len_1 = len_1 + 1

print(label_1_sum / len_1)

In [44]:
# list of the centers
clouds = []

for d in range(10):
    # Finding all the indexes from Y_train_out realted to the current d in loop
    d_indexs = np.where(Y_train == d)[0]
    # cloud_d is an array of all images related to the current d
    cloud_d = X_train[d_indexs]
    # Calculating the mean of each cloud
    center_d = np.mean(cloud_d, axis=0)
    # print(center_d)
    # add each mean vector to clouds
    clouds.append(center_d)

# print(clouds)

def classify(new_vector):
    distances = [LNG.norm(new_vector-center) for center in clouds]
    # print(distances)
    return np.argmin(distances)

new_vector = X_test[100]
classify(new_vector)

6